# 1.0 Imports e Leitura dos Dados

In [35]:
import holidays
import joblib

import pandas as pd
import numpy as np


from xgboost import XGBRegressor

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, r2_score, make_scorer

## 1.1 Leitura dos Dados

In [36]:
df = pd.read_csv('../data/cleaned_data_for_modelling.csv')

df["date"] = pd.to_datetime(df["date"])

# 2.0 Feature Engineering

Nessa etapa, que eu considero a mais crucial da modelagem, eu crio as features que nos permitirão modelar o problema em questão, que é a previsão do DAU.

Tendo em mente que a única feature que nós teremos no momento da previsão será **category**, e que as restantes precisarão ser atrasadas, pois não teremos nenhuma delas no momento da previsão (daily ratings, daily reviews, etc), eu pensei em modelar o problema como uma série temporal, de forma que para prever quantos usuários teremos no dia seguinte, usamos os dados de 7 dias para trás (1 semana). Sendo assim, o principal input para o modelo será o próprio DAU do aplicativo para dias anteriores ao que queremos prever, visto que teremos esses dados no momento da previsão.

Também fiz duas features relacionadas a datas que podem trazer picos de utilização, como feriados nacionais e finais de semana.

In [38]:
df = df.sort_values(by=["appId", "date"])

def lagged_features(df, lags=7):

    lagged_data = []

    for app in df["appId"].unique():
        app_data = df[df["appId"] == app].copy()
        for lag in range(1, lags + 1):
            app_data[f"dauReal_t-{lag}"] = app_data["dauReal"].shift(lag)
        app_data["dauReal_t"] = app_data["dauReal"]
        lagged_data.append(app_data)
    
    return pd.concat(lagged_data)

df_lagged = lagged_features(df, lags=7)

# Dropo as linhas com valores nulos, visto que faltam datas na nossa janela temporal
df_lagged = df_lagged.dropna()

In [39]:
# Checa se a data é um final de semana
df_lagged["is_weekend"] = df_lagged["date"].dt.weekday.isin([5, 6])

# Checa se a data é um feriado brasileiro
brazil_holidays = holidays.Brazil(years=2024)
df_lagged["is_brazilian_holiday"] = df_lagged["date"].isin(brazil_holidays)

/tmp/ipykernel_55820/1790318904.py:6: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df_lagged["is_brazilian_holiday"] = df_lagged["date"].isin(brazil_holidays)


# 3.0 Encoding

Nesta seção eu faço o encoding da coluna **category** para formato numérico.

In [40]:
label_encoder = LabelEncoder()

df_lagged['category'] = label_encoder.fit_transform(df_lagged['category'])

joblib.dump(label_encoder, "../models/label_encoder.joblib")

['../models/label_encoder.joblib']

# 4.0 Modelagem

In [41]:
df_lagged = df_lagged[
    [
        'appId','date','dauReal_t-1', 'dauReal_t-2', 'dauReal_t-3', 
        'dauReal_t-4', 'dauReal_t-5', 'dauReal_t-6', 
        'dauReal_t-7', 'category', 'is_weekend', 
        'is_brazilian_holiday', 'dauReal_t'
    ]
]

In [42]:
df_lagged.head()

,appId,date,dauReal_t-1,dauReal_t-2,dauReal_t-3,dauReal_t-4,dauReal_t-5,dauReal_t-6,dauReal_t-7,category,is_weekend,is_brazilian_holiday,dauReal_t
7,com.app.10626,2024-03-30,275086.0,287584.0,247458.0,259606.0,264645.0,206984.0,223700.0,1,True,False,236586.0
8,com.app.10626,2024-03-31,236586.0,275086.0,287584.0,247458.0,259606.0,264645.0,206984.0,1,True,False,195633.0
9,com.app.10626,2024-04-01,195633.0,236586.0,275086.0,287584.0,247458.0,259606.0,264645.0,1,False,False,298509.0
10,com.app.10626,2024-04-02,298509.0,195633.0,236586.0,275086.0,287584.0,247458.0,259606.0,1,False,False,293935.0
11,com.app.10626,2024-04-03,293935.0,298509.0,195633.0,236586.0,275086.0,287584.0,247458.0,1,False,False,281329.0


## 4.1 Train Test Split

Escolhi treinar os modelos utilizando a Time-Series Cross Validation, uma forma de entendermos como eles estão performando utilizando as janelas temporais para tentar prever o futuro próximo.

In [43]:
X = df_lagged[['dauReal_t-1', 'dauReal_t-2', 'dauReal_t-3', 'dauReal_t-4', 'dauReal_t-5', 'dauReal_t-6','dauReal_t-7', 'category', 'is_weekend', 'is_brazilian_holiday']]
y = df_lagged['dauReal_t']

n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits)

## 4.2 Modelos

Numa primeira iteração, utilizei uma combinação de três modelos que considero essenciais para um comparativo.

**Regressão Linear**:
- Simples, ideal para estabelecer um baseline.

**Random Forest**:
- Um modelo que utiliza a técnica de bagging (prevalece a previsão com mais votações dos modelos fracos) e geralmente fornece resultados muito robustos.

**XGBoost**
- Modelo que utiliza a técnica de boosting (modelos fracos são treinados sequencialmente para gerar um mais forte) e também fornece resultados robustos, além de ser extremamente eficiente.

In [44]:
model_reports = {}

# Dicionário com os modelos que serão testados
models = {
    "Linear Regression": LinearRegression(),
    "XGBoost": XGBRegressor(n_estimators=100, random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42)
}

for model_name, model in models.items():
    print(f"Treinando {model_name}...")
    
    mae_scores = []
    r2_scores = []

    for fold, (train_index, test_index) in enumerate(tscv.split(X)):
        
        # Divido os dados em treino e teste para esse Fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Treina o modelo
        model.fit(X_train, y_train)
        
        # Faz a previsão e calcula as métricas
        y_pred = model.predict(X_test)

        mae_score = mean_absolute_error(y_test, y_pred)
        mae_scores.append(mae_score)
        
        r2 = r2_score(y_test, y_pred)
        r2_scores.append(r2)

        print(f"  Fold {fold + 1}/{n_splits}... | MAE: {mae_score} | R2: {r2}")
    
    model_reports[model_name] = {
        "Mean MAE": np.mean(mae_scores),
        "Std MAE": np.std(mae_scores),
        "Mean R²": np.mean(r2_scores),
        "Std R²": np.std(r2_scores),
    }

    print(f"  Metrics for {model_name}:")
    print(f"    Mean MAE: {np.mean(mae_scores):.2f}, Std MAE: {np.std(mae_scores):.2f}")
    print(f"    Mean R²: {np.mean(r2_scores):.2f}, Std R²: {np.std(r2_scores):.2f}\n")

Treinando Linear Regression...
  Fold 1/5... | MAE: 41612.010915892184 | R2: 0.9902596839390214
  Fold 2/5... | MAE: 118134.61580416061 | R2: 0.9786088719105785
  Fold 3/5... | MAE: 39290.659382855214 | R2: 0.9427719156521275
  Fold 4/5... | MAE: 59791.88642221178 | R2: 0.9690514243541916
  Fold 5/5... | MAE: 36400.74035353994 | R2: 0.9943490624349015
  Metrics for Linear Regression:
    Mean MAE: 59045.98, Std MAE: 30656.31
    Mean R²: 0.98, Std R²: 0.02

Treinando XGBoost...
  Fold 1/5... | MAE: 59399.74171798851 | R2: 0.9755801729092635
  Fold 2/5... | MAE: 236630.30348198322 | R2: 0.9223029088370256
  Fold 3/5... | MAE: 22390.264546226463 | R2: 0.9436339053857424
  Fold 4/5... | MAE: 50622.24515755251 | R2: 0.9616022285270205
  Fold 5/5... | MAE: 43760.8937005321 | R2: 0.9785421155577025
  Metrics for XGBoost:
    Mean MAE: 82560.69, Std MAE: 77999.80
    Mean R²: 0.96, Std R²: 0.02

Treinando Random Forest...
  Fold 1/5... | MAE: 49425.9983196456 | R2: 0.97538341268552
  Fold 2/5

## 4.3 Hyperparameter Fine Tuning

In [47]:
def mae_scorer(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

scorer = make_scorer(
    mae_scorer, 
    greater_is_better=False
)

# Defino o espaço de busca para cada Hiperparâmetro
search_space = {
    "n_estimators": (50, 300),
    "max_depth": (2, 10),
    "learning_rate": (0.01, 0.3, "log-uniform"),
    "subsample": (0.5, 1.0, "uniform"),
    "colsample_bytree": (0.5, 1.0, "uniform"),
    "gamma": (0, 10), 
    "reg_alpha": (0, 1.0), 
    "reg_lambda": (0, 1.0), 
}

xgb = XGBRegressor(random_state=42)

opt = BayesSearchCV(
    estimator=xgb,
    search_spaces=search_space,
    cv=tscv,
    scoring=scorer, 
    n_iter=20,   
    random_state=42,
    verbose=2,
    n_jobs=-1
)

opt.fit(X, y)

print("Best Parameters:", opt.best_params_)
print("Best MAE:", -opt.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END colsample_bytree=0.705051979426657, gamma=7, learning_rate=0.2387586688716479, max_depth=5, n_estimators=218, reg_alpha=0.4141186324855385, reg_lambda=0.350931334899144, subsample=0.8697521170952103; total time=   0.4s
[CV] END colsample_bytree=0.705051979426657, gamma=7, learning_rate=0.2387586688716479, max_depth=5, n_estimators=218, reg_alpha=0.4141186324855385, reg_lambda=0.350931334899144, subsample=0.8697521170952103; total time=   0.4s
[CV] END colsample_bytree=0.705051979426657, gamma=7, learning_rate=0.2387586688716479, max_depth=5, n_estimators=218, reg_alpha=0.4141186324855385, reg_lambda=0.350931334899144, subsample=0.8697521170952103; total time=   0.5s
[CV] END colsample_bytree=0.705051979426657, gamma=7, learning_rate=0.2387586688716479, max_depth=5, n_estimators=218, reg_alpha=0.4141186324855385, reg_lambda=0.350931334899144, subsample=0.8697521170952103; total time=   0.5s
[CV] END colsample_bytree=0.7

In [49]:
best_params = opt.best_params_

final_model = XGBRegressor(**best_params)

final_model.fit(X, y)

joblib.dump(final_model, "../models/best_xgboost_model.joblib")

['../models/best_xgboost_model.joblib']